In [1]:
from tensorflow import keras 
import numpy as np 
import sklearn

2023-07-13 17:46:25.425008: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-13 17:46:25.459858: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-13 17:46:25.460372: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 17:46:26.131806: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD, Adam, Adadelta, RMSprop
import keras.backend as K
# Train-Test
from sklearn.model_selection import train_test_split
# Scaling data
from sklearn.preprocessing import StandardScaler
# Classification Report
from sklearn.metrics import classification_report

from keras.utils import to_categorical


In [3]:
# load dataset
df = pd.read_csv("/localscratch/df_preprocessed.csv")
df.shape

(1677867, 24)

In [4]:
df.keys()

Index(['Type', 'PT_l0', 'PT_l1', 'PT_l2', 'met', 'metPhi', 'm_l0l1', 'm_l0l2',
       'm_l1l2', 'm_lll', 'mT_l0l1', 'mT_l0l2', 'mT_l1l2', 'delEta_l0l1',
       'delEta_l0l2', 'delEta_l1l2', 'delPhi_l0l1', 'delPhi_l0l2',
       'delPhi_l1l2', 'delR_l0l1', 'delR_l0l2', 'delR_l1l2', 'sumPT', 'dRmin'],
      dtype='object')

In [5]:
x = df.drop('Type', axis=1)
x.shape

(1677867, 23)

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = pd.DataFrame(sc.fit_transform(x))

Y = df['Type'].values
print(Y)

y_cat = to_categorical(Y, num_classes=3)
print(y_cat)
print(x.values)

[1 1 1 ... 2 2 2]
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]
[[-0.47057967 -0.14251873  1.34747411 ...  0.10632183  0.20617413
  22.43056483]
 [-0.47057967 -0.82510717 -0.33154205 ...  0.49399187 -0.8793167
   5.29246475]
 [-0.47057967 -0.53112573  1.06903262 ...  0.43830518 -0.12956395
   4.95497724]
 ...
 [-0.39784018  0.33142598 -0.47308185 ... -0.29732863 -0.24189585
  -0.34822359]
 [-0.39784018  0.33142598 -0.64368563 ... -0.48316901 -0.31297517
  -0.34822359]
 [-0.39784018  0.33142598 -0.77305988 ...  0.74213032 -0.36687688
  -0.34822359]]


In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y_cat, test_size=0.2)
print(y_train)

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [8]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(Y), y=Y)
print(class_weight)

[5.59882475 4.76806281 0.3828979 ]


In [9]:
class_weight = {0: 5.59882475, 
                1: 4.76806281, 
                2: 0.3828979}

In [10]:
callback = keras.callbacks.EarlyStopping(
    monitor="loss",
    patience=3,
    verbose=0,
    mode="auto",
)

In [11]:
model = Sequential()
model.add(Dense(23, input_shape = (23,), activation = "swish"))
model.add(Dropout(0.2))
model.add(Dense(64, activation = "swish"))
model.add(Dropout(0.2))
model.add(Dense(64, activation = "swish"))
model.add(Dropout(0.2))
model.add(Dense(32, activation = "swish"))
model.add(Dropout(0.2))
model.add(Dense(32, activation = "swish"))
model.add(Dropout(0.2))
model.add(Dense(16, activation = "swish"))
model.add(Dropout(0.2))
model.add(Dense(16, activation = "swish"))
model.add(Dropout(0.2))
model.add(Dense(3, activation = "softmax"))
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 23)                552       
                                                                 
 dropout (Dropout)           (None, 23)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                1536      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 32)                2

In [ ]:
model.fit(x_train, y_train, verbose=1, epochs=200, class_weight = class_weight, callbacks=[callback])

Epoch 1/200
 3848/41947 [=>............................] - ETA: 1:03 - loss: 0.9714 - accuracy: 0.4939

In [ ]:
pred_train= model.predict(x_train)

scores = model.evaluate(x_train, y_train, verbose=0)

print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   

 

pred_test= model.predict(x_test)

scores2 = model.evaluate(x_test, y_test, verbose=0)

print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))    


In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(x_test)
y_pred_class = np.argmax(y_pred, axis=1)
print(y_pred)
y_test_class = np.argmax(y_test, axis=1)
#confusion_matrix(y_test_class, y_pred_class)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_class, y_pred_class))

In [ ]:
# Creating  a confusion matrix,which compares the y_test and y_pred
print(y_test[:,0])
print(y_pred[:,0])
cm = confusion_matrix(y_test[:,0], y_pred[:,0])

# Creating a dataframe for a array-formatted Confusion matrix,so it will be easy for plotting.
cm_df = pd.DataFrame(cm,
                     index = ['SETOSA','VERSICOLR','VIRGINICA'], 
                     columns = ['SETOSA','VERSICOLR','VIRGINICA'])

#Plotting the confusion matrix
plt.figure(figsize=(5,4))
sns.heatmap(cm_df, annot=True)
plt.title('Confusion Matrix')
plt.ylabel('Actal Values')
plt.xlabel('Predicted Values')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def show_values(pc, fmt="%.2f", **kw):
    '''
    Heatmap with text in each cell with matplotlib
    '''
    from itertools import izip
    pc.update_scalarmappable()
    ax = pc.get_axes()
    #ax = pc.axes# FOR LATEST MATPLOTLIB
    #Use zip BELOW IN PYTHON 3
    for p, color, value in izip(pc.get_paths(), pc.get_facecolors(), pc.get_array()):
        x, y = p.vertices[:-2, :].mean(0)
        if np.all(color[:3] > 0.5):
            color = (0.0, 0.0, 0.0)
        else:
            color = (1.0, 1.0, 1.0)
        ax.text(x, y, fmt % value, ha="center", va="center", color=color, **kw)


def cm2inch(*tupl):
    '''
    Specify figure size in centimeter in matplotlib
    Source: https://stackoverflow.com/a/22787457/395857
    By gns-ank
    '''
    inch = 2.54
    if type(tupl[0]) == tuple:
        return tuple(i/inch for i in tupl[0])
    else:
        return tuple(i/inch for i in tupl)


def heatmap(AUC, title, xlabel, ylabel, xticklabels, yticklabels, figure_width=40, figure_height=20, correct_orientation=False, cmap='RdBu'):

    # Plot it out
    fig, ax = plt.subplots()    
    #c = ax.pcolor(AUC, edgecolors='k', linestyle= 'dashed', linewidths=0.2, cmap='RdBu', vmin=0.0, vmax=1.0)
    c = ax.pcolor(AUC, edgecolors='k', linestyle= 'dashed', linewidths=0.2, cmap=cmap)

    # put the major ticks at the middle of each cell
    ax.set_yticks(np.arange(AUC.shape[0]) + 0.5, minor=False)
    ax.set_xticks(np.arange(AUC.shape[1]) + 0.5, minor=False)

    # set tick labels
    #ax.set_xticklabels(np.arange(1,AUC.shape[1]+1), minor=False)
    ax.set_xticklabels(xticklabels, minor=False)
    ax.set_yticklabels(yticklabels, minor=False)

    # set title and x/y labels
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)      

    # Remove last blank column
    plt.xlim( (0, AUC.shape[1]) )

    # Turn off all the ticks
    ax = plt.gca()    
    for t in ax.xaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False
    for t in ax.yaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False

    # Add color bar
    plt.colorbar(c)

    # Add text in each cell 
    show_values(c)

    # Proper orientation (origin at the top left instead of bottom left)
    if correct_orientation:
        ax.invert_yaxis()
        ax.xaxis.tick_top()       

    # resize 
    fig = plt.gcf()
    #fig.set_size_inches(cm2inch(40, 20))
    #fig.set_size_inches(cm2inch(40*4, 20*4))
    fig.set_size_inches(cm2inch(figure_width, figure_height))



def plot_classification_report(classification_report, title='Classification report ', cmap='RdBu'):
    lines = classification_report.split('\n')

    classes = []
    plotMat = []
    support = []
    class_names = ['VH', 'WWW', 'bkg']
    for line in lines[2 : (len(lines) - 2)]:
        t = line.strip().split()
        if len(t) < 2: continue
        classes.append(t[0])
        v = [float(x) for x in t[1: len(t) - 1]]
        support.append(int(t[-1]))
        print(v)
        plotMat.append(v)

    print('plotMat: {0}'.format(plotMat))
    print('support: {0}'.format(support))

    xlabel = 'Metrics'
    ylabel = 'Classes'
    xticklabels = ['Precision', 'Recall', 'F1-score']
    yticklabels = ['{0} ({1})'.format(class_names[idx], sup) for idx, sup  in enumerate(support)]
    figure_width = 25
    figure_height = len(class_names) + 7
    correct_orientation = False
    heatmap(np.array(plotMat), title, xlabel, ylabel, xticklabels, yticklabels, figure_width, figure_height, correct_orientation, cmap=cmap)


def main():
    sampleClassificationReport = """ precision    recall  f1-score   support

           0       0.20      0.93      0.33     19641
           1       0.28      0.76      0.41     23482
           2       0.99      0.61      0.75    292451

"""


    plot_classification_report(sampleClassificationReport)
    plt.savefig('/localscratch/classif_report.png', dpi=200, format='png', bbox_inches='tight')
    plt.close()

if __name__ == "__main__":
    main()
    #cProfile.run('main()') # if you want to do some profiling


In [ ]:
FPR, TPR, TH = roc_curve(y_test, y_score, sample_weight=class_weights, pos_label=1)
roc_auc = roc_auc_score(y_test, y_score)